# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\natal\OneDrive\Desktop\Classwork\NU-VIRT-DATA-PT-10-2023-U-LOLC\02-Homework\06-Python-APIs\Instructions\Starter_Code\output_data\cities.csv")

# Display sample data
city_data_df

##note: for some reason the relative path was not working so I had to use the absolute path

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,42.37,69,15,16.02,TF,1709926756
1,1,tyukhtet,56.5389,89.3011,2.05,98,73,5.70,RU,1709926469
2,2,walvis bay,-22.9575,14.5053,67.15,82,14,10.36,NaN,1709926779
3,3,tiksi,71.6872,128.8694,-25.85,100,59,2.66,RU,1709926769
4,4,ziro,27.6333,93.8333,42.71,87,57,3.09,IN,1709927026
...,...,...,...,...,...,...,...,...,...,...
566,566,cheremshan,54.6667,51.5000,20.01,96,100,8.32,RU,1709927099
567,567,yelizovo,53.1873,158.3805,12.34,56,40,6.71,RU,1709926787
568,568,woodward,36.4337,-99.3904,34.83,93,100,20.71,US,1709927100
569,569,george town,5.4112,100.3354,82.33,88,20,2.30,MY,1709926614


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
humidity_df = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      scale=0.5,
                                      color='Humidity',
                                      hover_cols=['City'],
                                      tiles='OSM',
                                      frame_width=700,
                                      frame_height=500)

# Display the map
humidity_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27)
    & (city_data_df["Max Temp"] > 21)
    & (city_data_df["Wind Speed"] < 6.5)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,juuka,63.2333,29.2500,22.44,94,100,2.73,FI,1709927028
92,92,gadzhiyevo,69.2551,33.3362,26.08,97,100,6.26,RU,1709926798
98,98,qamdo,31.1667,97.2333,22.24,45,100,5.97,CN,1709927037
120,120,kirensk,57.7853,108.1119,22.96,94,100,4.00,RU,1709927039
169,169,zaozersk,69.4005,32.4476,25.16,97,100,5.41,RU,1709927045
203,203,dryden,49.7833,-92.7503,23.29,63,20,5.75,CA,1709926491
259,259,la ronge,55.1001,-105.2842,21.45,42,0,2.30,CA,1709927059
264,264,khovd,48.0056,91.6419,22.10,58,53,5.10,MN,1709926777
267,267,winkler,49.1817,-97.9410,23.18,73,53,5.75,CA,1709927060
451,451,mynamaki,60.7446,22.0243,25.11,83,100,4.41,FI,1709927084


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,juuka,FI,63.2333,29.2500,94,
92,gadzhiyevo,RU,69.2551,33.3362,97,
98,qamdo,CN,31.1667,97.2333,45,
120,kirensk,RU,57.7853,108.1119,94,
169,zaozersk,RU,69.4005,32.4476,97,
203,dryden,CA,49.7833,-92.7503,63,
259,la ronge,CA,55.1001,-105.2842,42,
264,khovd,MN,48.0056,91.6419,58,
267,winkler,CA,49.1817,-97.9410,73,
451,mynamaki,FI,60.7446,22.0243,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
hotel_df["Hotel Name"] = ""

# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey": geoapify_key, "limit": 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"][
            "name"
        ]
    except (KeyError, IndexError):
		
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(
        f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}"
    )

# Display sample data
hotel_df

Starting hotel search
juuka - nearest hotel: Petra
gadzhiyevo - nearest hotel: Оазис
qamdo - nearest hotel: 银杏商务宾馆
kirensk - nearest hotel: Ивушка
zaozersk - nearest hotel: Северное сияние
dryden - nearest hotel: Holiday Inn Express
la ronge - nearest hotel: La Ronge Hotel and Suites
khovd - nearest hotel: Khovd hotel
winkler - nearest hotel: Quality Inn & Suites Winkler
mynamaki - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
polyarnyy - nearest hotel: No hotel found
dalandzadgad - nearest hotel: Хан Уул буудал


,City,Country,Lat,Lng,Humidity,Hotel Name
20,juuka,FI,63.2333,29.2500,94,Petra
92,gadzhiyevo,RU,69.2551,33.3362,97,Оазис
98,qamdo,CN,31.1667,97.2333,45,银杏商务宾馆
120,kirensk,RU,57.7853,108.1119,94,Ивушка
169,zaozersk,RU,69.4005,32.4476,97,Северное сияние
203,dryden,CA,49.7833,-92.7503,63,Holiday Inn Express
259,la ronge,CA,55.1001,-105.2842,42,La Ronge Hotel and Suites
264,khovd,MN,48.0056,91.6419,58,Khovd hotel
267,winkler,CA,49.1817,-97.9410,73,Quality Inn & Suites Winkler
451,mynamaki,FI,60.7446,22.0243,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      scale=0.5,
                                      color='Humidity',
                                      hover_cols=['Hotel Name'],
                                      tiles='OSM',
                                      frame_width=700,
                                      frame_height=500)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)